# Example of AI Agent

#### The Notebook demonstrates how to create a simple AI Agent that uses vLLM & OpenAI API to communicate with LLM Model.

### Importing Required Libraries

In [1]:
import os
from openai import OpenAI
import json
from vllm import SamplingParams

INFO 07-19 09:43:11 [__init__.py:248] Automatically detected platform rocm.


## Deploying the LLM using vLLM

Start deploying the LLM (/deepseek-ai/deepseek-llm-7b-chat) using vLLM in the Jupyter notebook:

### Start the vLLM server 

Open a new tab in this Jypyter server, click on the terminal icon to open a new terminal, then copy the following command to launch the vLLM server:

```bash
HIP_VISIBLE_DEVICES=0 vllm serve /home/user/Models/deepseek-ai/deepseek-llm-7b-chat \
        --gpu-memory-utilization 0.9 \
        --swap-space 16 \
        --disable-log-requests \
        --dtype float16 \
        --max-model-len 2048 \
        --tensor-parallel-size 1 \
        --host 0.0.0.0 \
        --port 3000 \
        --num-scheduler-steps 10 \
        --max-num-seqs 128 \
        --max-num-batched-tokens 2048 \
        --max-model-len 2048 \
        --distributed-executor-backend "mp"
```

#### Configure the BASE_URL & MODEL_PATH to input to AI Agent 

In [3]:
BASE_URL = f"http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"

#### Create OpenAI Client

In [4]:
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)

#### Create an AI Agent that takes OpenAI Client & Text Input and follows pre-defined Instructions to output the response

In [5]:
class AI_AGENT:
    def __init__(self, client, MODEL_PATH):
        self.base_url = BASE_URL
        self.model_path = MODEL_PATH

    def parse_email(self, email_text):
        response = client.chat.completions.create(
            model=self.model_path,
            temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                Yor are an Agent that helps in scheduling meetings.
                Your job is to extracts Email ID's and Meeting Duration.
                You should return :
                1. List of email id's of participants (comma-separated).
                2. Meeting duration in minutes.
                3. Time constraints (e.g., 'next week').
                If the List of email id's of participants are just names, then append @amd.com at the end and return. 
                Return as json with 'participants', 'time_constraints' & 'meeting_duration'.
                Stricty follow the instructions. Strictly return dict with participents email id's, time constraints & meeting duration in minutes only. 
                Don not add any other instrctions or information. 
                
                Email: {email_text}
                
                """
            }]
        )
        return json.loads(response.choices[0].message.content)

In [6]:
ai_agent = AI_AGENT(client, MODEL_PATH)

In [7]:
email = """
Hi team, let's meet about AI Project next week for half hour. 
Attendees: Ajith(ajith@amd.com), Prakash(prakash@gmail.com), Sirra(sirra@gmail.com). 
"""
meeting_json = ai_agent.parse_email(email)
meeting_json

{'participants': 'ajith@amd.com,prakash@gmail.com,sirra@gmail.com',
 'time_constraints': 'next week',
 'meeting_duration': 30}

In [8]:
meeting_json

{'participants': 'ajith@amd.com,prakash@gmail.com,sirra@gmail.com',
 'time_constraints': 'next week',
 'meeting_duration': 30}

In [9]:
email = """
Hi Team, let's meet about AI Project next tuesday for an hour. 
Attendees: Ajith, Prakash & Sirra. 
"""
meeting_json = ai_agent.parse_email(email.lower())
meeting_json

{'participants': 'ajith@amd.com,prakash@amd.com,sirra@amd.com',
 'time_constraints': 'next tuesday',
 'meeting_duration': 60}